In [109]:
import pandas as pd
import numpy as np
import torch
import math
import torch.nn as nn
from tqdm import tqdm
from torch.nn import LayerNorm
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from utils import *
import torch.nn.functional as F

In [3]:
node_coords = pd.read_csv("data/sioux/SiouxFalls_node.tntp", sep='\t')
node_coords_arr = np.array(node_coords[['X', 'Y']])

sioux = create_network_df(network_name="SiouxFalls")
T_0, C = prepare_network_data(sioux)

directory = "/home/podozerovapo/traffic_assignment/data/sioux/uncongested"

inputs = []
outputs = []
metadata = []

for filename in sorted(os.listdir(directory)):
    if filename.endswith(".pkl"):
        filepath = os.path.join(directory, filename)
        
        with open(filepath, 'rb') as f:
            data_pair = pickle.load(f)
            
            inputs.append(data_pair['input'])
            outputs.append(data_pair['output'])
            metadata.append(data_pair.get('metadata', None))

input_matrices = np.array(inputs)  # [num_samples, num_nodes, num_nodes]
output_matrices = np.array(outputs)  # [num_samples, num_nodes, num_nodes]


In [ ]:
class FeatureEmbedding(nn.Module):
    def __init__(self, input_size, embedding_size=32):
        super(FeatureEmbedding, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, embedding_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.network(x)

class SharedEncoderComponents(nn.Module):
    def __init__(self, node_feat_size, n_heads, dropout=0.1):
        super().__init__()
        self.node_feat_size = node_feat_size
        self.n_heads = n_heads
        self.head_dim = node_feat_size // n_heads
        
        self.q_linear = nn.Linear(node_feat_size, node_feat_size)
        self.k_linear = nn.Linear(node_feat_size, node_feat_size)
        self.v_linear = nn.Linear(node_feat_size, node_feat_size)
        
        self.out_ffn = nn.Sequential(
            nn.Linear(node_feat_size, node_feat_size),
            nn.ReLU(),
            nn.Linear(node_feat_size, node_feat_size)
        )
        
        self.layer_norm = nn.LayerNorm(node_feat_size)
        self.dropout = nn.Dropout(dropout)

class VEncoderLayer(nn.Module):
    def __init__(self, node_feat_size, edge_feat_size, n_heads, dropout=0.1):
        super().__init__()
        self.shared = SharedEncoderComponents(node_feat_size, n_heads, dropout)
        self.n_heads = n_heads
        self.head_dim = node_feat_size // n_heads
        self.edge_weight_ffn = nn.Sequential(
            nn.Linear(2 * node_feat_size, edge_feat_size),
            nn.ReLU(),
            nn.Linear(edge_feat_size, n_heads),
            nn.Sigmoid()
        )
        
        self.ffn = nn.Sequential(
            nn.Linear(node_feat_size, node_feat_size * 2),
            nn.ReLU(),
            nn.Linear(node_feat_size * 2, node_feat_size),
            nn.Dropout(dropout)
        )
        
    def forward(self, x, adj_mask):
        batch_size, num_nodes, _ = x.size()

        q = self.shared.q_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        k = self.shared.k_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        v = self.shared.v_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        src_nodes = x.unsqueeze(2).expand(-1, -1, num_nodes, -1)
        dst_nodes = x.unsqueeze(1).expand(-1, num_nodes, -1, -1)
        node_pairs = torch.cat([src_nodes, dst_nodes], dim=-1)
        beta = self.edge_weight_ffn(node_pairs).permute(0, 3, 1, 2)
        
        adj_mask = adj_mask.unsqueeze(1)
        scores = scores * adj_mask * beta
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.shared.dropout(attn_weights)
        
        output = torch.matmul(attn_weights, v)
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, num_nodes, self.shared.node_feat_size)
        
        output = self.shared.out_ffn(output)
        output = self.shared.dropout(output)
        output = self.shared.layer_norm(x + output)
        
        ffn_output = self.ffn(output)
        output = self.shared.layer_norm(output + ffn_output)
        
        return output

class REncoderLayer(nn.Module):
    def __init__(self, node_feat_size, edge_feat_size, n_heads, dropout=0.1):
        super().__init__()
        self.shared = SharedEncoderComponents(node_feat_size, n_heads, dropout)
        self.n_heads = n_heads
        self.head_dim = node_feat_size // n_heads
        self.edge_feat_transform = nn.Linear(edge_feat_size, n_heads)
        
        self.ffn = nn.Sequential(
            nn.Linear(node_feat_size, node_feat_size * 2),
            nn.ReLU(),
            nn.Linear(node_feat_size * 2, node_feat_size),
            nn.Dropout(dropout)
        )
        
    def forward(self, x, adj_mask, edge_features):
        batch_size, num_nodes, _ = x.size()
        
        q = self.shared.q_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        k = self.shared.k_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        v = self.shared.v_linear(x).view(batch_size, num_nodes, self.n_heads, self.head_dim)
        
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        edge_weights = self.edge_feat_transform(edge_features).permute(0, 3, 1, 2)
    
        adj_mask = adj_mask.unsqueeze(1)
        scores = scores * adj_mask * torch.sigmoid(edge_weights)
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.shared.dropout(attn_weights)
        
        output = torch.matmul(attn_weights, v)
        output = output.transpose(1, 2).contiguous()
        output = output.view(batch_size, num_nodes, self.shared.node_feat_size)
        
        output = self.shared.out_ffn(output)
        output = self.shared.dropout(output)
        output = self.shared.layer_norm(x + output)
        
        ffn_output = self.ffn(output)
        output = self.shared.layer_norm(output + ffn_output)
        
        return output

class VEncoder(nn.Module):
    def __init__(self, node_feat_size, edge_feat_size, n_layers=3, n_heads=4, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            VEncoderLayer(node_feat_size, edge_feat_size, n_heads, dropout)
            for _ in range(n_layers)
        ])
        
    def forward(self, x, v_adj_mask):
        for layer in self.layers:
            x = layer(x, v_adj_mask)
        return x

class REncoder(nn.Module):
    def __init__(self, node_feat_size, edge_feat_size, n_layers=3, n_heads=4, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            REncoderLayer(node_feat_size, edge_feat_size, n_heads, dropout)
            for _ in range(n_layers)
        ])
        
    def forward(self, x, adj_mask, edge_features):
        for layer in self.layers:
            x = layer(x, adj_mask, edge_features)
        return x

class DualGraphEncoder(nn.Module):
    def __init__(self, node_feat_size, edge_feat_size,
                 v_layers=3, r_layers=3, n_heads=4, dropout=0.1):
        super().__init__()
        self.vencoder = VEncoder(node_feat_size, edge_feat_size, v_layers, n_heads, dropout)
        self.rencoder = REncoder(node_feat_size, edge_feat_size, r_layers, n_heads, dropout)

    def forward(self, node_features, real_adj_mask, virtual_adj_mask, edge_features):
        v_encoded = self.vencoder(node_features, virtual_adj_mask)
        r_encoded = self.rencoder(node_features, real_adj_mask, edge_features)
        return v_encoded, r_encoded

class TrafficAssignmentModel(nn.Module):
    def __init__(self, num_nodes, node_feat_size=32, edge_feat_size=2,
                 v_layers=2, r_layers=2, n_heads=4, dropout=0.1):
        super().__init__()
        
        self.feature_preprocessor = FeatureEmbedding(input_size=num_nodes + 2, embedding_size=32)
        
        self.dual_encoder = DualGraphEncoder(node_feat_size, edge_feat_size, 
                                           v_layers, r_layers, n_heads, dropout)
        
        self.flow_predictor = nn.Sequential(
            nn.Linear(4 * node_feat_size, node_feat_size),
            nn.ReLU(),
            nn.Linear(node_feat_size, 1),
            nn.ReLU()
        )

    def forward(self, node_features, real_adj_mask, virt_adj_mask, edge_features):
        
        node_features = self.feature_preprocessor(node_features)
        v_encoded, r_encoded = self.dual_encoder(node_features, real_adj_mask, virt_adj_mask, edge_features)
        combined = torch.cat([v_encoded, r_encoded], dim=-1)
        src_nodes = combined.unsqueeze(2).expand(-1, -1, combined.size(1), -1)
        dst_nodes = combined.unsqueeze(1).expand(-1, combined.size(1), -1, -1)
        pair_features = torch.cat([src_nodes, dst_nodes], dim=-1)

        flows = self.flow_predictor(pair_features).squeeze(-1)
        flows = flows * real_adj_mask
        
        return flows

In [5]:
coords_for_concat = np.repeat(np.expand_dims(node_coords_arr, 0), repeats=input_matrices.shape[0],axis=0)

In [6]:
X = np.concatenate([input_matrices, coords_for_concat], axis=2)
X.shape

(4096, 24, 26)

In [7]:
capacities = np.repeat(np.expand_dims(C, 0), axis=0, repeats=input_matrices.shape[0])
free_flow_times = np.repeat(np.expand_dims(T_0, 0), axis=0, repeats=input_matrices.shape[0])

In [ ]:
class TrafficDataset(Dataset):
    def __init__(self, X_data, capacity_data, free_flow_data, flows_data):
        self.data = []
        for x, cap, fft, fl in zip(X_data, capacity_data, free_flow_data, flows_data):
            od_matrix = x[:, :-2]
            coordinates = x[:, -2:]

            od_mean = od_matrix.mean()
            od_std = od_matrix.std() + 1e-8
            od_matrix = (od_matrix - od_mean) / od_std

            coord_mean = coordinates.mean(axis=0)
            coord_std = coordinates.std(axis=0) + 1e-8
            coordinates = (coordinates - coord_mean) / coord_std

            x_normalized = np.concatenate([od_matrix, coordinates], axis=1)

            cap = (cap - cap.mean()) / (cap.std() + 1e-8)
            fft = (fft - fft.mean()) / (fft.std() + 1e-8)
            fl = (fl - fl.mean()) / (fl.std() + 1e-8)

            self.data.append({
                'od_matrix': torch.FloatTensor(od_matrix),
                'coordinates': torch.FloatTensor(coordinates),
                'capacity': torch.FloatTensor(cap),
                'free_flow': torch.FloatTensor(fft),
                'flows': torch.FloatTensor(fl),
                'x_normalized' : torch.FloatTensor(x_normalized)
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        real_adj_mask = (item['capacity'] > 0).float()
        virtual_adj = item['od_matrix'].abs()

        edge_features = torch.stack([
            item['capacity'],
            item['free_flow']
        ], dim=-1)

        agg_demand = item['od_matrix'].sum(dim=1, keepdim=True)
        node_features = torch.cat([item['coordinates'], agg_demand], dim=1)

        return {
            'node_features': node_features,
            'real_adj_mask': real_adj_mask,
            'edge_features': edge_features,
            'virtual_adj': virtual_adj,
            'target_flows': item['flows'],
            'x_normalized' : item['x_normalized']
        }

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=100, lr=1e-3, patience=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3)
    criterion = nn.MSELoss()
    
    best_loss = float('inf')
    epochs_no_improve = 0
    train_history = []
    val_history = []
    
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            node_feat = batch['x_normalized'].to(device)
            real_mask = batch['real_adj_mask'].to(device)
            edge_feat = batch['edge_features'].to(device)
            virt_adj = batch['virtual_adj'].to(device)
            targets = batch['target_flows'].to(device)
            
            optimizer.zero_grad()
            outputs = model(node_feat, real_mask, virt_adj, edge_feat)
            loss = criterion(outputs[real_mask.bool()], targets[real_mask.bool()])
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item() * node_feat.size(0)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                node_feat = batch['x_normalized'].to(device)
                real_mask = batch['real_adj_mask'].to(device)
                edge_feat = batch['edge_features'].to(device)
                virt_adj = batch['virtual_adj'].to(device)
                targets = batch['target_flows'].to(device)
                
                outputs = model(node_feat, real_mask, virt_adj, edge_feat)
                
                loss = criterion(outputs[real_mask.bool()], targets[real_mask.bool()])
                val_loss += loss.item() * node_feat.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        val_loss = val_loss / len(val_loader.dataset)
        train_history.append(train_loss)
        val_history.append(val_loss)
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')
        print(f'LR: {optimizer.param_groups[0]["lr"]:.2e}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f'Early stopping after {epoch+1} epochs')
                break
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_history, label='Train Loss')
    plt.plot(val_history, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training History')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()
    
    model.load_state_dict(torch.load('best_model.pth'))
    return model

In [ ]:
def model_predict(model, data_loader, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = model.to(device)
    model.eval()

    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Making predictions'):
            node_feat = batch['x_normalized'].to(device)
            real_mask = batch['real_adj_mask'].to(device)
            edge_feat = batch['edge_features'].to(device)
            virt_adj = batch['virtual_adj'].to(device)
            targets = batch['target_flows'].to(device)

            outputs = model(node_feat, real_mask, virt_adj, edge_feat)

            mask = real_mask.bool()
            all_predictions.append(outputs[mask].cpu().numpy())
            all_targets.append(targets[mask].cpu().numpy())
            break

    predictions = np.concatenate(all_predictions)
    targets = np.concatenate(all_targets)

    return predictions, targets

In [107]:
ans = model_predict(model, train_loader)

Making predictions:   0%|          | 0/52 [00:00<?, ?it/s]


In [108]:
print(ans[0], ans[1], sep='\n')

[0.48642147 0.9556702  0.5360018  ... 2.403835   2.7833183  1.0279554 ]
[0.4724989  0.96577114 0.63058925 ... 2.6740074  2.8549805  0.9699092 ]


In [106]:
n_nodes = 24
X_data = X
capacity_data = capacities
free_flow_data = free_flow_times
flows_data = output_matrices

full_dataset = TrafficDataset(X_data, capacity_data, free_flow_data, flows_data)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

model = TrafficAssignmentModel(24)

# Запуск обучения
trained_model = train_model(
    model,
    train_loader,
    val_loader,
    num_epochs=100,
    lr=1e-3,
    patience=10
)

# Сохранение финальной модели
torch.save(trained_model.state_dict(), 'final_model.pth')

  1%|          | 1/100 [00:03<05:46,  3.50s/it]

Epoch 1/100
Train Loss: 1.7905 | Val Loss: 0.8613
LR: 1.00e-03


  2%|▏         | 2/100 [00:06<05:42,  3.49s/it]

Epoch 2/100
Train Loss: 0.6359 | Val Loss: 0.3121
LR: 1.00e-03


  3%|▎         | 3/100 [00:10<05:47,  3.59s/it]

...

Epoch 98/100
Train Loss: 0.0286 | Val Loss: 0.0275
LR: 1.25e-04


 99%|█████████▉| 99/100 [05:39<00:03,  3.46s/it]

Epoch 99/100
Train Loss: 0.0285 | Val Loss: 0.0275
LR: 6.25e-05


100%|██████████| 100/100 [05:43<00:00,  3.43s/it]

Epoch 100/100
Train Loss: 0.0284 | Val Loss: 0.0273
LR: 6.25e-05


In [7]:
from tqdm import tqdm

In [103]:
model = FlowDistributionModel(num_nodes, hidden_dim=32).cuda()

In [104]:
def train(model, dataloader, epochs=5):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for batch in dataloader:
            OD, C, T0, targets = batch.values()
            # Move data to GPU
            OD, C, T0, targets = OD.cuda(), C.cuda(), T0.cuda(), targets.cuda()
            
            optimizer.zero_grad()
            preds = model(OD, C, T0)
            loss = F.mse_loss(preds, targets)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(dataloader):.4f}")

train(model, dataloader)

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (552x3 and 16x32)

In [80]:
model = FlowDistributionModel(num_nodes=num_nodes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm(range(100)):
    for batch in dataloader:
        OD_batch, C_batch, T0_batch, target_flows_batch = batch.values()

        optimizer.zero_grad()
        
        pred_flows = model(OD_batch, C_batch, T0_batch)
        loss = loss_fn(pred_flows, target_flows_batch, OD_batch, C_batch)
        
        loss.backward()
        optimizer.step()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!